# Calculating Equity of Access to Opportunities

In this notebook, we will estimate the *change* in the level of access to opportunities (in the form of employment opportunities) accross our study region by comparing three different potential transit scenarios with a baseline "business as usual" scenario.

The basic workflow for this process is as follows:

1. Load travel time and demand matrices for each scenario
2. Estimate or specify an impedance function for the measure of access to opportunities. In this workbook we will use an impedance function based on observed model behaviour.
3. Calculate the level of access to employment for each zone in our study area, and calculate this difference from the business as usual function.
4. Create population weighted summaries for various population groups to compare the impact of each scenario on these groups, and visualize these results.

To get started, lets load our required Python modules and specify the location of various datasets we will need, including the matrices folder containing the travel time and demand matrices, the opportunities folder with the employment specifciation, and the folder containing the link created between dissemination areas (census zones) and transport analysis zones from the model.

In [ ]:
import os
import pandas as pd
import altair as alt
import numpy as np
from scipy.stats import norm

# Replace with your data folder paths
mx_folder = r"data/matrices/input"
counts_folder = r"data/counts/input"
link_folder = r"data/counts/interim"
output_folder = r"data/counts/output"

Next, we load in our four travel time matrices. Since these can be quite large, it's useful to specify specific data types for each column to make loading the data a bit less memory intensive. We do that using the `dtype` keyword.

In [ ]:
dtypes = {'i':int, 'j':int, 'travel_time':float, 'demand':float}
mx_BAU = pd.read_csv(os.path.join(mx_folder, 'bau_times_flows.csv'), dtype=dtypes)
mx_A = pd.read_csv(os.path.join(mx_folder, 'scenario_A_times_flows.csv'), dtype=dtypes)
mx_B = pd.read_csv(os.path.join(mx_folder, 'scenario_B_times_flows.csv'), dtype=dtypes)
mx_C = pd.read_csv(os.path.join(mx_folder, 'scenario_C_times_flows.csv'), dtype=dtypes)

## Estimating an Admittance Function

Now that we have travel times and demand, we need to estimate or provide a defined impedance function to weight the travel times against to get a more useful measure of access to opportunities. 

In the code below, we are going to use an approach which uses the overall characteristics of the distribution of trip lenghts in our dataset to generate a smooth curve. This requires the following steps:

1. Calculate the mean and standard deviation of the travel times accross all origin-destination points, weighted by the demand (number of trips taken)
2. Use this mean and standard deviation to generate a cumulative normal distribution curve, and use 1 minus this curve as the impedance function

Let's start by generating the demand-weighted mean and standard deviation of time in the matrices. Here we will use the business as usual scenario as our baseline for all admittance/impedance functions.

In [ ]:
mean = np.average(mx_BAU.travel_time, weights=mx_BAU.demand)
std = np.sqrt(np.average((mx_BAU.travel_time - mean)**2, weights=mx_BAU.demand))
print(mean, std)

Our impedance function is now easily specified as `1 - norm.cdf(<travel_time>, loc=mean, scale=std)`. To get a sense of how well our normal curve fits the actual distribution, we'll generate a histogram of our PDF and plot that alongside our theoretical curve.

In [ ]:
# First we generate a histogram as our PDF
hist_vals, hist_bins=np.histogram(mx_BAU.travel_time, 200, weights=mx_BAU.demand, density=False)
# Normalize them between 0 and 1
hist_vals /= np.sum(hist_vals)
imp = pd.DataFrame({'pdf': hist_vals, 'cdf':  np.cumsum(hist_vals), 'imp':  1-np.cumsum(hist_vals), 'travel_time': hist_bins[1:]})
# Create a theoretical impedence
imp['norm'] = 1-norm.cdf(imp['travel_time'], loc=mean, scale=std)

# Plot the data - 
real = alt.Chart(imp).mark_line(clip=True, color="#0D0A45", size=2).encode(
    alt.X('travel_time:Q', title="Travel time (min)", scale=alt.Scale(domain=(1, 180))),
    alt.Y('imp:Q', title="Admittance")
)

# Plot the model - dashed line
model = alt.Chart(imp).mark_line(color='#6D0D4A', clip=True, strokeDash=[5], size=2).encode(
    alt.X('travel_time:Q'),
    alt.Y('norm:Q')
)

(real+model).properties(
    width = 300,
    height=200
).configure(font='Roboto').configure_axis(grid=False).configure_view(strokeWidth=0)

With this function estimated, we can now apply it to the travel time for each origin-destination pair:

In [ ]:
mx_BAU['impedance'] = 1-norm.cdf(mx_BAU.travel_time, loc=mean, scale=std)
mx_A['impedance'] = 1-norm.cdf(mx_A.travel_time, loc=mean, scale=std)
mx_B['impedance'] = 1-norm.cdf(mx_B.travel_time, loc=mean, scale=std)
mx_C['impedance'] = 1-norm.cdf(mx_C.travel_time, loc=mean, scale=std)

## Calculating Access to Opportunities
The final ingredient we need is the actual opportunities to access. For this workbook, we are considering projected employment numbers, however any set of opportunities will do. We do this with the following steps for each scenario matrix:

1. Load the opportunities, and merge them with the matrix
2. Multiply the calculated admittance/impedance by the number of opportunities
3. Group and sum for each origin `i`

In [ ]:
ops = pd.read_csv(os.path.join(counts_folder, 'taz_employment.csv'))

acc_BAU = pd.merge(mx_BAU, ops, left_on='i', right_on='taz_id')
acc_BAU['emp_imp_BAU'] = acc_BAU.impedance * acc_BAU.employment
access_BAU = acc_BAU[['i', 'emp_imp_BAU']].groupby('i', as_index=False).sum()

acc_A = pd.merge(mx_A, ops, left_on='i', right_on='taz_id')
acc_A['emp_imp_A'] = acc_A.impedance * acc_A.employment
access_A = acc_A[['i', 'emp_imp_A']].groupby('i', as_index=False).sum()

acc_B = pd.merge(mx_B, ops, left_on='i', right_on='taz_id')
acc_B['emp_imp_B'] = acc_B.impedance * acc_B.employment
access_B = acc_B[['i', 'emp_imp_B']].groupby('i', as_index=False).sum()

acc_C = pd.merge(mx_C, ops, left_on='i', right_on='taz_id')
acc_C['emp_imp_C'] = acc_C.impedance * acc_C.employment
access_C = acc_C[['i', 'emp_imp_C']].groupby('i', as_index=False).sum()

Since we are interested in the *change* in access from the original scenario, let's calcualte the percent change for each of our three intervention scenarios compared to the business as usual scenario. To do this we must merge the datasets together and calculate various percent differences. We'll also write the output so we can generate a map of the percent change spatially if we'd like.

In [ ]:
delta = pd.merge(access_BAU, access_A, on='i')
delta = pd.merge(delta, access_B, on='i')
delta = pd.merge(delta, access_C, on='i')
delta['delta_A'] = 100 * (delta['emp_imp_A'] - delta['emp_imp_BAU'])/(delta['emp_imp_BAU'])
delta['delta_B'] = 100 * (delta['emp_imp_B'] - delta['emp_imp_BAU'])/(delta['emp_imp_BAU'])
delta['delta_C'] = 100 * (delta['emp_imp_C'] - delta['emp_imp_BAU'])/(delta['emp_imp_BAU'])
delta.to_csv(os.path.join(output_folder, 'access_change_by_taz.csv'), index=False)

## Equity of Access to Opportunities
The final step is to find some distributive summaries accross population groups. Using the percent change in access we calculated for our scenarios, we can do weighted summaries for population groups. To do this we:
1. Load and join the link file between TAZ and dissemination area generated by areal apportionment.
2. Load and join the demographic counts by dissemination area
3. Generate fractional counts of each demographic group in each TAZ
4. Conduct a population weighted sum for each taz by each group of interest

Let's do the first two steps together

In [ ]:
taz_da = pd.read_csv(os.path.join(link_folder, 'taz_da_link.csv'), dtype={'DAUID': int, 'taz_id': int, 'frac_da_in_taz': float})
da_demo = pd.read_csv(os.path.join(counts_folder, 'da_census_profile.csv'))
taz_da_acc = pd.merge(taz_da, delta, left_on='taz_id', right_on='i')
taz_da_acc = pd.merge(taz_da_acc, da_demo, on='DAUID')
taz_da_acc.head()

Next, we use a bit of Pandas wizardry to multiply the fraction of a given da population in a TAZ by each demographic count, rename the columns for consistency, and then put them together with our original dataset to get a table containing our access changes, and the count of each demographic group in each intersected area. We will perform the calculation for three population groups: Total population, visible minorities, and low income individuals.

In [ ]:
# Minus 1-11 columns
da_acc = taz_da_acc[taz_da_acc.columns[-11:-1]].multiply(taz_da_acc['frac_da_in_taz'], axis="index")
da_acc.columns = [f"f_{c}" for c in da_acc.columns]
da_acc = pd.concat([taz_da_acc[['delta_A', 'delta_B', 'delta_C']], da_acc], axis=1)
display(da_acc.head())
da_acc['A_pop_2016'] = (da_acc['f_pop_2016']/da_acc['f_pop_2016'].sum()) * da_acc['delta_A']
da_acc['B_pop_2016'] = (da_acc['f_pop_2016']/da_acc['f_pop_2016'].sum()) * da_acc['delta_B']
da_acc['C_pop_2016'] = (da_acc['f_pop_2016']/da_acc['f_pop_2016'].sum()) * da_acc['delta_C']

da_acc['A_vm_minority'] = (da_acc['f_vm_minority']/da_acc['f_vm_minority'].sum()) * da_acc['delta_A']
da_acc['B_vm_minority'] = (da_acc['f_vm_minority']/da_acc['f_vm_minority'].sum()) * da_acc['delta_B']
da_acc['C_vm_minority'] = (da_acc['f_vm_minority']/da_acc['f_vm_minority'].sum()) * da_acc['delta_C']

da_acc['A_income_lim'] = (da_acc['f_income_lim']/da_acc['f_income_lim'].sum()) * da_acc['delta_A']
da_acc['B_income_lim'] = (da_acc['f_income_lim']/da_acc['f_income_lim'].sum()) * da_acc['delta_B']
da_acc['C_income_lim'] = (da_acc['f_income_lim']/da_acc['f_income_lim'].sum()) * da_acc['delta_C']
display(da_acc.head())



Finally, we want to produce a visualization that shows how these scenarios differ for various population groups. To do this, we will keep only the data we are interested in, change the table format by using the `melt()` function, and plot it in Altair. We do a bit of hacky string manipulation here to get our labels correct for plotting.

In [ ]:
to_plot = da_acc[['A_pop_2016', 'B_pop_2016', 'C_pop_2016', 'A_vm_minority', 'B_vm_minority', 'C_vm_minority', 'A_income_lim', 'B_income_lim', 'C_income_lim']]
pretty_names = {'income_lim': "Low Income (LIM)", 'pop_2016': 'Total Population', 'vm_minority': "Visible Minority"}
melted = to_plot.melt().groupby('variable', as_index=False).sum()
melted['scenario'] = melted.variable.str[0]
melted['demographic'] = melted.variable.str[2:]
melted['demo_name'] = melted.demographic.map(pretty_names)
melted

Now we assemble a plot showing the comparative differences.

In [ ]:
bar = alt.Chart(melted).mark_bar().encode(
    alt.Y('scenario:N', title=None),
    alt.X('value:Q', title='Percent Change in Access'),
    alt.Color('scenario:N', title='Scenario'),
    alt.Row('demo_name:N', title='', sort=['Total Population'], spacing=35)
)

(bar).properties(
    title="Change in Access to Opportunities for SmartTrack Scenarios",
    width=600,
    height=80
).configure(font='Roboto').configure_axis(grid=False).configure_view(strokeWidth=0).configure_title(fontSize=18)